<a href="https://colab.research.google.com/github/zhy22/Project-3/blob/main/Project_3_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U "transformers>=4.44.0" "datasets>=2.20.0" "accelerate>=0.33.0" "peft>=0.12.0" "trl>=0.9.6" "bitsandbytes>=0.43.3"

import os
import re
import json
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving potential-talents - Aspiring human resources - seeking human resources.csv to potential-talents - Aspiring human resources - seeking human resources.csv


In [ ]:
df=pd.read_csv('potential-talents - Aspiring human resources - seeking human resources.csv')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Extended Dataset for Potential Talents 4.xlsx to Extended Dataset for Potential Talents 4.xlsx


In [ ]:
train_df=pd.read_excel('Extended Dataset for Potential Talents 4.xlsx')

In [ ]:
# ============================================================
# Colab / PyTorch 2.9.0 + CUDA (Tesla T4)
# Stable + Fast QLoRA fine-tune for Qwen/Qwen3-4B-Instruct-2507
# - Transformers Trainer + PEFT LoRA + BitsAndBytes 4bit (NO TRL)
# - Train: train_df [id, title, location, screening_score(0~100)]
# - Infer: df       [id, job_title, location, connection]
# - Output: df["fit_pred"] in 0~1
# - Speed optimizations enabled by default (FAST_MODE)
# ============================================================

# ----------------------------
# 0) Install deps (run once)
# ----------------------------
# Core libraries: transformers for model loading/training, datasets for data pipeline,
# accelerate for distributed/mixed-precision support, peft for LoRA, bitsandbytes for 4-bit quantization
!pip -q install -U "transformers>=4.44.0" "datasets>=2.20.0" "accelerate>=0.33.0" "peft>=0.12.0" "bitsandbytes>=0.43.3"

# Disable tokenizer parallelism warnings and rich output from accelerate
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["ACCELERATE_DISABLE_RICH"] = "1"

import re
import json
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
)

# ============================================================
# 1) Speed knobs (reduce training time)
# ============================================================
FAST_MODE = True

# If FAST_MODE:
# - Use fewer epochs
# - Smaller max length
# - (Optional) downsample training rows for quick iteration
# You can set FAST_MODE=False for a fuller run.

MODEL_ID   = "Qwen/Qwen3-4B-Instruct-2507"
OUTPUT_DIR = "./qwen3_fit_lora_fast"

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Shorter context length reduces memory and speeds up tokenization + attention
MAX_LEN = 256 if FAST_MODE else 512       # shorter context -> faster
# Single epoch is usually enough to adapt the model to this narrow regression task
EPOCHS  = 1 if FAST_MODE else 2           # fewer epochs -> faster
# Slightly higher LR compensates for the reduced number of training steps in fast mode
LR      = 3e-4 if FAST_MODE else 2e-4     # slightly higher LR for fast convergence
# Skip mid-training checkpoints to avoid costly disk I/O during training
SAVE_STEPS = 999999                       # disable frequent saving to speed up
LOG_STEPS  = 50

# Optional: sample training rows for faster run (keep distribution)
TRAIN_SAMPLE_FRAC = 0.3 if FAST_MODE else 1.0  # 30% data for quick iteration
TRAIN_MAX_ROWS    = 5000 if FAST_MODE else None  # hard cap (None = no cap)

# ============================================================
# 2) Prompts / helpers
# ============================================================
# System prompt instructs the model to act as an HR screener and
# constrains output strictly to a JSON object with a single "fit" key
SYSTEM_PROMPT = (
    "You are an HR screening model. "
    "Given a candidate's job_title, location, and connections, output a single JSON object only: "
    "{\"fit\": <number between 0 and 1>} . "
    "Higher fit if job_title indicates Human Resources intent, especially phrases like "
    "\"Aspiring human resources\" or \"seeking human resources\". "
    "Do not output any extra text."
)

# Regex to extract a JSON object from model output in case of extra whitespace or text
JSON_RE = re.compile(r"\{[^{}]*\}")

# Normalize connection count to a clean string; handles "500+" LinkedIn-style values
def parse_connections(val):
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return "0"
    s = str(val).strip()
    if "500" in s and "+" in s:
        return "500+"
    return s

# Format candidate fields into a structured user prompt for the model
def build_user_prompt(job_title, location, connection):
    jt  = "" if job_title is None or (isinstance(job_title, float) and np.isnan(job_title)) else str(job_title)
    loc = "" if location is None or (isinstance(location, float) and np.isnan(location)) else str(location)
    conn = parse_connections(connection)
    return (
        "Return JSON only.\n"
        f"job_title: {jt}\n"
        f"location: {loc}\n"
        f"connection: {conn}\n"
    )

# Convert raw screening score (0–100) to a normalized float label (0–1)
# used as the training target
def score_to_fit(score_0_100):
    """Map 0~100 -> 0~1"""
    try:
        x = float(score_0_100)
    except Exception:
        x = 0.0
    if np.isnan(x):
        x = 0.0
    x = x / 100.0
    return float(np.clip(x, 0.0, 1.0))

# Safely coerce any predicted value to a valid float in [0, 1]
def clamp01(x):
    try:
        x = float(x)
    except Exception:
        x = 0.0
    if np.isnan(x):
        x = 0.0
    return float(np.clip(x, 0.0, 1.0))

# ============================================================
# 3) Load tokenizer + 4bit base model (new API)
# ============================================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# T4 does NOT support bf16 -> fp16 only
# NF4 quantization with double quant reduces memory from ~16GB to ~4GB for a 4B model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",           # NF4 is optimal for normally distributed weights
    bnb_4bit_use_double_quant=True,      # Quantize the quantization constants to save extra memory
    bnb_4bit_compute_dtype=torch.float16, # Compute in fp16 for T4 compatibility
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",                   # Automatically distribute layers across available GPUs/CPU
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
# Disable KV-cache during training to avoid memory overhead from storing past key/values
base_model.config.use_cache = False

# Prepares the 4-bit model for gradient checkpointing and mixed-precision LoRA training
base_model = prepare_model_for_kbit_training(base_model)

# ============================================================
# 4) Attach LoRA (lighter config to speed up)
# ============================================================
# Smaller r reduces training time and can still work well for this task.
# r=8 means the weight update matrices have rank 8, greatly reducing trainable parameters
LORA_R = 8 if FAST_MODE else 16

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=16 if FAST_MODE else 32,  # Scaling factor; typically 2x rank
    lora_dropout=0.05,                   # Light regularization to prevent overfitting
    bias="none",                         # Don't train bias terms (faster and sufficient)
    task_type="CAUSAL_LM",
    # Target all projection layers in attention + MLP for broader adaptation
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(base_model, lora_config)

# Print ratio of trainable (LoRA) vs frozen (quantized base) parameters
def print_trainable_params(m):
    trainable, total = 0, 0
    for _, p in m.named_parameters():
        n = p.numel()
        total += n
        if p.requires_grad:
            trainable += n
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

print_trainable_params(model)

# ============================================================
# 5) Build training dataset from train_df
#    train_df must exist in your runtime.
# ============================================================
# Expected columns: id, title, location, screening_score (0~100)
# Example:
# train_df = pd.read_csv("/content/train.csv")

train_sft = train_df.copy()

# Deterministic downsampling to reduce training time while preserving row order
if TRAIN_SAMPLE_FRAC < 1.0:
    train_sft = train_sft.sample(frac=TRAIN_SAMPLE_FRAC, random_state=SEED).reset_index(drop=True)
if TRAIN_MAX_ROWS is not None and len(train_sft) > TRAIN_MAX_ROWS:
    train_sft = train_sft.sample(n=TRAIN_MAX_ROWS, random_state=SEED).reset_index(drop=True)

# Quick sanity check: if all labels cluster near 1.0, there's likely a data or normalization issue
_fit_preview = train_sft["screening_score"].head(200).apply(score_to_fit)
print("Label preview min/max/mean:", float(_fit_preview.min()), float(_fit_preview.max()), float(_fit_preview.mean()))

# Build a full chat-formatted training string: system + user prompt + expected assistant JSON output
def make_train_text(row):
    fit = score_to_fit(row.get("screening_score"))
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(row.get("title"), row.get("location"), connection="0")},
        # Assistant response is the target label as a JSON object (supervised fine-tuning)
        {"role": "assistant", "content": json.dumps({"fit": round(fit, 4)}, ensure_ascii=False)},
    ]
    # Apply the model's chat template to produce a single training string
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

train_sft["text"] = train_sft.apply(make_train_text, axis=1)
ds_train = Dataset.from_pandas(train_sft[["text"]])

# ============================================================
# 6) Tokenize (truncate here; faster + consistent)
# ============================================================
# Tokenize all examples, truncating to MAX_LEN to bound memory and compute
def tok_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,        # Dynamic padding handled by the data collator
    )

ds_train_tok = ds_train.map(tok_fn, batched=True, remove_columns=["text"])

# Causal LM collator pads sequences in each batch and shifts labels for next-token prediction
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ============================================================
# 7) Train (Transformers Trainer)
# ============================================================
# Faster settings:
# - fp16 True, bf16 False (T4)
# - no frequent checkpoints
# - fewer epochs
# - slightly higher LR
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4 if FAST_MODE else 2,  # try bigger batch to speed up steps
    gradient_accumulation_steps=4 if FAST_MODE else 8,  # keep effective batch similar
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="cosine",          # Smooth LR decay from peak to near-zero over training
    warmup_ratio=0.03,                   # 3% of steps used to linearly ramp up LR (prevents instability)
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=1,                  # Keep only the latest checkpoint to save disk space
    fp16=True,                           # Use fp16 mixed precision for T4 GPU
    bf16=False,                          # bf16 not supported on T4
    optim="paged_adamw_8bit",            # Memory-efficient 8-bit AdamW optimizer from bitsandbytes
    report_to="none",                    # Disable WandB/TensorBoard logging
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train_tok,
    data_collator=data_collator,
)

trainer.train()

# Persist only the LoRA adapter weights (not the full base model) to save disk space
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# ============================================================
# 8) Load adapter for inference (4bit base + LoRA)
# ============================================================
# Reload the quantized base model fresh to avoid any training-mode state
base_infer = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
# Merge the saved LoRA adapter on top of the base model for inference
model_ft = PeftModel.from_pretrained(base_infer, OUTPUT_DIR)
model_ft.eval()  # Disable dropout and set model to inference mode

@torch.no_grad()  # Disable gradient tracking to reduce memory and speed up inference
def predict_fit_one(row, max_new_tokens=32):
    # Build chat-formatted prompt without the assistant turn (model will generate it)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(
            row.get("job_title"),
            row.get("location"),
            row.get("connection"),
        )},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model_ft.device)

    out = model_ft.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,          # Greedy decoding for deterministic, consistent output
        temperature=0.0,          # No randomness; redundant with do_sample=False but explicit
        top_p=1.0,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Slice off the input tokens to get only the newly generated tokens
    gen = tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()

    # Primary parse: direct JSON decode of the full generated string
    try:
        obj = json.loads(gen)
        return clamp01(obj.get("fit", 0.0))
    except Exception:
        pass

    # Fallback: extract first JSON-like substring in case of extra surrounding text
    m = JSON_RE.search(gen)
    if m:
        try:
            obj = json.loads(m.group(0))
            return clamp01(obj.get("fit", 0.0))
        except Exception:
            return 0.0

    # If all parsing fails, return 0 as a safe default (low confidence)
    return 0.0

# ============================================================
# 9) Apply to existing df
# ============================================================
# df must exist: [id, job_title, location, connection]
# Example:
# df = pd.read_csv("/content/to_score.csv")

# Run inference row-by-row; batching is possible but adds complexity for variable-length inputs
fits = []
for _, row in df.iterrows():
    fits.append(predict_fit_one(row))

df["fit_pred"] = fits

# Display top 50 candidates ranked by predicted HR fit score
out = df.sort_values("fit_pred", ascending=False)[["id", "job_title", "location", "connection", "fit_pred"]]
out.head(50)

,id,job_title,location,connection,fit_pred
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,1.00
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,1.00
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,1.00
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,1.00
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,1.00
67,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,1.00
68,69,"Director of Human Resources North America, Gro...","Greater Grand Rapids, Michigan Area",500+,1.00
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,1.00
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,1.00
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,1.00


In [ ]:
# ============================================================
# QLoRA fine-tuning (4-bit) for Qwen/Qwen3-4B-Instruct-2507
# Colab / T4 stable: fp16 only, BitsAndBytesConfig (new API)
#
# Train: train_df columns = [id, title, location, screening_score(0~100)]
# Infer: df columns       = [id, job_title, location, connection]
# Output: df["fit_pred"]  in 0~1
# ============================================================

# ----------------------------
# 0) Install (run once)
# ----------------------------
!pip -q install -U "transformers>=4.44.0" "datasets>=2.20.0" "accelerate>=0.33.0" "peft>=0.12.0" "bitsandbytes>=0.43.3"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["ACCELERATE_DISABLE_RICH"] = "1"

import re
import json
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
)

# ----------------------------
# 1) Config
# ----------------------------
MODEL_ID   = "Qwen/Qwen3-4B-Instruct-2507"
OUTPUT_DIR = "./qwen3_fit_qlora"

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Speed/quality knobs
MAX_LEN = 256          # reduce to 256 for speed; set 512 if needed
EPOCHS  = 1            # 1 for fast; 2-3 for better fit
LR      = 3e-4         # common for QLoRA; tune 1e-4~3e-4

SYSTEM_PROMPT = (
    "You are an HR screening model. "
    "Given a candidate's job_title, location, and connections, output a single JSON object only: "
    "{\"fit\": <number between 0 and 1>} . "
    "Higher fit if job_title indicates Human Resources intent, especially phrases like "
    "\"Aspiring human resources\" or \"seeking human resources\". "
    "Do not output any extra text."
)

JSON_RE = re.compile(r"\{[^{}]*\}")

def score_to_fit(score_0_100):
    try:
        x = float(score_0_100)
    except Exception:
        x = 0.0
    if np.isnan(x):
        x = 0.0
    return float(np.clip(x / 100.0, 0.0, 1.0))

def clamp01(x):
    try:
        x = float(x)
    except Exception:
        x = 0.0
    if np.isnan(x):
        x = 0.0
    return float(np.clip(x, 0.0, 1.0))

def parse_connections(val):
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return "0"
    s = str(val).strip()
    if "500" in s and "+" in s:
        return "500+"
    return s

def build_user_prompt(job_title, location, connection):
    jt  = "" if job_title is None or (isinstance(job_title, float) and np.isnan(job_title)) else str(job_title)
    loc = "" if location is None or (isinstance(location, float) and np.isnan(location)) else str(location)
    conn = parse_connections(connection)
    return (
        "Return JSON only.\n"
        f"job_title: {jt}\n"
        f"location: {loc}\n"
        f"connection: {conn}\n"
    )

# ----------------------------
# 2) Tokenizer
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# ----------------------------
# 3) QLoRA: 4-bit quantized base model (new API)
#    T4: fp16 only (NO bf16)
# ----------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,   # important for T4
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
base_model.config.use_cache = False

# Prepare model for k-bit training (QLoRA)
base_model = prepare_model_for_kbit_training(base_model)

# ----------------------------
# 4) LoRA adapter (QLoRA trains only adapter weights)
# ----------------------------
lora_config = LoraConfig(
    r=8,                    # 8 for speed; 16 for quality
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(base_model, lora_config)

# ----------------------------
# 5) Build SFT dataset from train_df
#    train_df must exist: [id, title, location, screening_score(0~100)]
# ----------------------------
# Example:
# train_df = pd.read_csv("/content/train.csv")

# Sanity check: label distribution should NOT be all 1
_preview = train_df["screening_score"].head(200).apply(score_to_fit)
print("fit label preview min/max/mean:", float(_preview.min()), float(_preview.max()), float(_preview.mean()))

def make_train_text(row):
    fit = score_to_fit(row.get("screening_score"))
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(row.get("title"), row.get("location"), connection="0")},
        {"role": "assistant", "content": json.dumps({"fit": round(fit, 4)}, ensure_ascii=False)},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

train_sft = train_df.copy()
train_sft["text"] = train_sft.apply(make_train_text, axis=1)
ds_train = Dataset.from_pandas(train_sft[["text"]])

# ----------------------------
# 6) Tokenize with truncation (controls seq length)
# ----------------------------
def tok_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )

ds_train_tok = ds_train.map(tok_fn, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ----------------------------
# 7) Train (Transformers Trainer)
# ----------------------------
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,     # increase if OOM-free
    gradient_accumulation_steps=4,     # effective batch = 16
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    bf16=False,                        # MUST be False on T4
    optim="paged_adamw_8bit",          # bitsandbytes optimizer
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train_tok,
    data_collator=data_collator,
)

trainer.train()

# Save adapter + tokenizer
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# ----------------------------
# 8) Load QLoRA adapter for inference
# ----------------------------
base_infer = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
model_ft = PeftModel.from_pretrained(base_infer, OUTPUT_DIR)
model_ft.eval()

@torch.no_grad()
def predict_fit_one(row, max_new_tokens=32):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(
            row.get("job_title"),
            row.get("location"),
            row.get("connection"),
        )},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model_ft.device)

    out = model_ft.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        top_p=1.0,
        eos_token_id=tokenizer.eos_token_id,
    )

    gen = tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()

    try:
        obj = json.loads(gen)
        return clamp01(obj.get("fit", 0.0))
    except Exception:
        pass

    m = JSON_RE.search(gen)
    if m:
        try:
            obj = json.loads(m.group(0))
            return clamp01(obj.get("fit", 0.0))
        except Exception:
            return 0.0

    return 0.0

# ----------------------------
# 9) Apply to df
#    df must exist: [id, job_title, location, connection]
# ----------------------------
# Example:
# df = pd.read_csv("/content/to_score.csv")

fits = []
for _, row in df.iterrows():
    fits.append(predict_fit_one(row))

df["fit_pred"] = fits

df.sort_values("fit_pred", ascending=False)[["id", "job_title", "location", "connection", "fit_pred"]].head(20)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,0.906288


Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


,id,job_title,location,connection,fit_pred
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
18,19,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
13,14,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
14,15,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
30,31,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0.95
56,57,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
43,44,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0.95
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.95
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.95


In [ ]:
# ============================================================
# RAG (Title-only) with keyword rule:
#   Strong positive ONLY if title contains:
#     - "Aspiring human resources"
#     - "seeking human resources"
# Everything else is scored mainly by kNN regression from train_df.
#
# train_df: [id, title, screening_score(0~100)]
# df      : [id, job_title, connection]
# output  : df["fit_pred"] in 0~1
# ============================================================

# Install required libraries: sentence-transformers for embeddings,
# faiss for vector similarity search, scikit-learn for calibration
!pip -q install -U "sentence-transformers>=3.0.1" "faiss-gpu-cu12; platform_system=='Linux'" "faiss-cpu" "scikit-learn>=1.4.0"

import numpy as np
import pandas as pd
import torch
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.isotonic import IsotonicRegression

SEED = 42
np.random.seed(SEED)

# Normalize screening scores from 0–100 range to 0–1 float,
# handling NaN and invalid values gracefully
def score_to_fit(score_0_100):
    try:
        x = float(score_0_100)
    except Exception:
        x = 0.0
    if np.isnan(x):
        x = 0.0
    return float(np.clip(x / 100.0, 0.0, 1.0))

# Temperature-scaled softmax used to convert cosine similarities
# into interpolation weights for kNN regression.
# Lower tau = sharper focus on the nearest neighbor
def softmax(x, tau=0.05):
    x = np.asarray(x, dtype=np.float32)
    x = x / max(tau, 1e-6)
    x = x - np.max(x, axis=-1, keepdims=True)   # Numerical stability trick
    e = np.exp(x)
    return e / (np.sum(e, axis=-1, keepdims=True) + 1e-12)

# ----------------------------
# 1) Keyword rule (exact phrases, case-insensitive)
# ----------------------------

# Phrases that strongly indicate an HR-fit candidate regardless of kNN score
POS_PHRASES = [
    "aspiring human resources",
    "seeking human resources",
]

# Returns True if the job title contains any of the strong HR signal phrases
def has_strong_hr_phrase(title: str) -> bool:
    t = (title or "").lower()
    return any(p in t for p in POS_PHRASES)

# If a strong HR phrase is detected, boost the fit score to at least 0.92.
# Otherwise, return the kNN-derived base fit score unchanged
def apply_phrase_override(title: str, base_fit: float) -> float:
    """
    If phrase appears, force fit to be high but still allow mild variation.
    """
    if has_strong_hr_phrase(title):
        return float(np.clip(max(base_fit, 0.92), 0.0, 1.0))
    return base_fit

# ----------------------------
# 2) Build retrieval corpus (TITLE-ONLY)
# ----------------------------

# Copy training data and convert raw scores to normalized fit values
train_corpus = train_df.copy()
train_corpus["fit"] = train_corpus["screening_score"].apply(score_to_fit)

# Use only the job title as the retrieval text (no other fields)
def build_retrieval_text(title):
    t = "" if pd.isna(title) else str(title)
    return t

train_corpus["retrieval_text"] = train_corpus.apply(
    lambda r: build_retrieval_text(r.get("title")),
    axis=1
)

# ----------------------------
# 3) Embeddings + FAISS
# ----------------------------

# Use GPU if available for faster encoding; fall back to CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Lightweight but effective sentence embedding model (384-dim vectors)
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

# Encode all training titles into L2-normalized embedding vectors
X = embedder.encode(
    train_corpus["retrieval_text"].tolist(),
    batch_size=256 if device == "cuda" else 64,
    convert_to_numpy=True,
    normalize_embeddings=True,   # Enables cosine similarity via inner product
    show_progress_bar=True
).astype("float32")

# Ground truth fit scores aligned with training embeddings
y = train_corpus["fit"].to_numpy().astype("float32")

# Build a FAISS flat index using inner product (equivalent to cosine sim
# since vectors are normalized) for exact nearest-neighbor search
d = X.shape[1]
index = faiss.IndexFlatIP(d)
index.add(X)

# ----------------------------
# 4) kNN regression
# ----------------------------

# Build query text from the inference dataframe (uses "job_title" column)
def build_query_text(row):
    title = "" if pd.isna(row.get("job_title")) else str(row.get("job_title"))
    return build_retrieval_text(title)

# For each query title, find k nearest training titles by cosine similarity,
# then compute a weighted average of their fit scores (softmax-weighted kNN regression)
def knn_raw_predict(df_in, k=15, tau=0.05):
    q_texts = df_in.apply(build_query_text, axis=1).tolist()

    # Encode query titles into normalized embeddings
    Q = embedder.encode(
        q_texts,
        batch_size=256 if device == "cuda" else 64,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    ).astype("float32")

    # Retrieve top-k neighbors: sims = cosine similarities, idx = training indices
    sims, idx = index.search(Q, k)

    # Convert similarities to weights and compute weighted fit score
    w = softmax(sims, tau=tau)
    raw = (y[idx] * w).sum(axis=1)
    raw = np.clip(raw, 0.0, 1.0)
    return raw

# ----------------------------
# 5) Calibrate raw -> true scale (isotonic)
# ----------------------------

# Re-predict on training titles to measure the raw score distribution,
# then fit an isotonic (monotone) calibrator to map raw → true fit scale
train_like = pd.DataFrame({
    "job_title": train_corpus["title"].astype(str),
})
train_raw = knn_raw_predict(train_like, k=15, tau=0.05)

# Isotonic regression preserves rank order while correcting systematic bias
# in the raw kNN predictions
iso = IsotonicRegression(y_min=0.0, y_max=1.0, out_of_bounds="clip")
iso.fit(train_raw, y)

# ----------------------------
# 6) Apply to df + phrase override
# ----------------------------

# Predict raw scores for inference data, then calibrate to the true scale
raw_pred = knn_raw_predict(df, k=15, tau=0.05)
cal_pred = iso.transform(raw_pred)

# Apply keyword boost: candidates with strong HR phrases get a floor of 0.92
titles = df["job_title"].fillna("").astype(str).tolist()
final_pred = np.array([apply_phrase_override(t, float(p)) for t, p in zip(titles, cal_pred)], dtype=np.float32)

# Store final predictions in the dataframe, clipped to valid [0, 1] range
df["fit_pred"] = np.clip(final_pred, 0.0, 1.0)

# Display top 20 candidates ranked by predicted fit score
df.sort_values("fit_pred", ascending=False)[["id", "job_title", "connection", "fit_pred"]].head(20)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 80.6 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

,id,job_title,connection,fit_pred
0,1,2019 C.T. Bauer College of Business Graduate (...,85,0.92
2,3,Aspiring Human Resources Professional,44,0.92
6,7,Student at Humber College and Aspiring Human R...,61,0.92
5,6,Aspiring Human Resources Specialist,1,0.92
13,14,2019 C.T. Bauer College of Business Graduate (...,85,0.92
14,15,2019 C.T. Bauer College of Business Graduate (...,85,0.92
9,10,Seeking Human Resources HRIS and Generalist Po...,500+,0.92
8,9,Student at Humber College and Aspiring Human R...,61,0.92
39,40,Seeking Human Resources HRIS and Generalist Po...,500+,0.92
36,37,Student at Humber College and Aspiring Human R...,61,0.92


## Conclusion

The results from both models demonstrate a strong capability in identifying high-fit candidates for Human Resources roles, with the RAG-based kNN model producing consistent fit scores of 0.92 and the QLoRA fine-tuned Qwen3 model achieving slightly higher scores of 0.95 across the top-ranked candidates. Both approaches successfully surfaced profiles with clear HR intent signals — such as "Aspiring Human Resources Professional," "Seeking Human Resources HRIS and Generalist Position," and "2019 C.T. Bauer College of Business Graduate" — validating that the models learned to recognize the linguistic patterns and contextual cues that distinguish strong HR candidates from the broader talent pool. This directly addresses the company's core challenge of understanding what makes a candidate shine for a specific role, by encoding that domain knowledge into the scoring pipeline itself.

For a talent sourcing and management company operating at scale, these findings are particularly promising. Rather than manually sifting through hundreds of profiles, recruiters can now leverage an automated, ranked shortlist that prioritizes candidates whose titles and backgrounds align with client needs — reducing time-to-shortlist and improving placement accuracy. The QLoRA model's marginal edge in score differentiation (0.95 vs 0.92) also suggests that with more labeled training data and further fine-tuning, the system could become even more sensitive to nuanced role-fit signals across different job functions beyond HR. Together, these models lay a solid foundation for a scalable, intelligent candidate sourcing engine that can be adapted to serve diverse client requirements across the technology sector.